In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

2025-12-19 00:15:27.488840: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("sentiment.csv", encoding="latin-1")
#Reading the CSV file

/tmp/ipykernel_24117/1164029311.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("sentiment.csv", encoding="latin-1")


In [4]:
df.dropna(axis=0,how="any")

,ProductName,ProductPrice,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,Super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,Awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,Fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,Useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,Fair,ok ok product,neutral
...,...,...,...,...,...,...
171374,mobspy 3.5mm Clip Microphone For Youtube | Col...,168.0,5.0,Best in the market!,its very awesome productby every video makers ...,positive
171375,mobspy 3.5mm Clip Microphone For Youtube | Col...,168.0,5.0,Must buy!,awesome product and clear sound just go it,positive
171376,mobspy 3.5mm Clip Microphone For Youtube | Col...,168.0,5.0,Perfect product!,good,positive
171377,mobspy 3.5mm Clip Microphone For Youtube | Col...,168.0,2.0,Slightly disappointed,lot of echo when recording,negative


In [7]:
df["Sentiment"]=df["Sentiment"].astype(str).str.lower().str.strip()
df = df[df["Sentiment"].isin(["positive", "negative"])]
df["texts"]=(df["Summary"].astype(str) + " " +df["Review"].astype(str))
df["texts"] = (
    df["texts"]
    .str.lower()
    .str.replace(r"[^a-z\s]", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
) #Basic cleaning
df=df[df["texts"].str.len()>0]
labels=df["Sentiment"].map({"negative":0,"positive":1}).astype("int32").values
texts=df["texts"].values

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157241 entries, 0 to 171378
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ProductName   157241 non-null  object
 1   ProductPrice  157241 non-null  object
 2   Rate          157241 non-null  object
 3   Review        157241 non-null  object
 4   Summary       157241 non-null  object
 5   Sentiment     157241 non-null  object
 6   texts         157241 non-null  object
dtypes: object(7)
memory usage: 9.6+ MB


In [9]:
vectorizer=tf.keras.layers.TextVectorization(max_tokens=10000,output_sequence_length=120,ngrams=2)
vectorizer.adapt(texts)
X=vectorizer(texts)

I0000 00:00:1766085330.695164   24117 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3772 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [10]:
model=tf.keras.Sequential([tf.keras.layers.Embedding(10000,64),
                           tf.keras.layers.GlobalMaxPooling1D(),
                           tf.keras.layers.Dense(64,activation="relu"),
                           tf.keras.layers.Dropout(0.5),
                           tf.keras.layers.Dense(1,activation="sigmoid")])

In [11]:
class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs=None):
        if logs["accuracy"]>0.99:
            self.model.stop_training=True

In [12]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [13]:
model.fit(X,labels,epochs=5,batch_size=32,validation_split=0.3,shuffle=True,callbacks=[callback()])

Epoch 1/5


2025-12-19 00:15:34.081074: I external/local_xla/xla/service/service.cc:163] XLA service 0x75afa8004630 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-19 00:15:34.081103: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-12-19 00:15:34.099547: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-19 00:15:34.239957: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


 101/3440 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6582 - loss: 0.6356

I0000 00:00:1766085335.222520   24166 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3417/3440 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9240 - loss: 0.1815

2025-12-19 00:15:40.726956: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_445', 16 bytes spill stores, 16 bytes spill loads



3440/3440 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9603 - loss: 0.1115 - val_accuracy: 0.9747 - val_loss: 0.0665
Epoch 2/5
3440/3440 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9774 - loss: 0.0677 - val_accuracy: 0.9744 - val_loss: 0.0660
Epoch 3/5
3440/3440 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9809 - loss: 0.0548 - val_accuracy: 0.9749 - val_loss: 0.0680
Epoch 4/5
3440/3440 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9838 - loss: 0.0457 - val_accuracy: 0.9739 - val_loss: 0.0734
Epoch 5/5
3440/3440 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9860 - loss: 0.0385 - val_accuracy: 0.9735 - val_loss: 0.0790


In [21]:
while True:
    text=input("Enter Text: ")
    x=vectorizer([text])
    pred=model.predict(x)[0][0]
    
    if pred>0.7:
        print("Postive Review Recived")
        print(pred)
        break
    if pred<0.3:
        print("Negtive Review Recived")
        print(pred)
        break
    else:
        print("Mixed Review Recived")
        print(pred)
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Negtive Review Recived
0.261299
